# 03. 출력 파서(Output Parsers)

<br>

### 출력파서(Output Parser)
- LangChain의 출력파서는 언어 모델(LLM)의 출력을 더 유용하고 구조화된 형태로 변환하는 중요한 컴포넌트

<br>

#### 출력파서의 역할
- LLM의 출력을 받아 더 적합한 형식으로 변환
- 구조화된 데이터 생성에 매우 유용
- LangChain 프레임워크에서 다양한 종류의 출력 데이터를 파싱하고 처리

<br>

#### 주요 특징
- **다양성** : LangChain은 많은 종류의 출력 파서를 제공
- **스트리밍 지원** : 많은 출력 파서들이 스트리밍을 지원
- **확장성** : 최소한의 모듈부터 복잡한 모듈까지 확장 가능한 인터페이스를 제공

<br>

#### 출력파서의 이점
- **구조화** : LLM의 자유 형식 텍스트 출력을 구조화된 데이터로 변환
- **일관성** : 출력 형식을 일관되게 유지하여 후속 처리를 용이
- **유연성** : 다양한 출력 형식(JSON, 리스트, 딕셔너리 등)으로 변환이 가능

<br>

#### 출력파서를 사용할 때와 사용하지 않을 때
- 아무런 출력파서를 사용하지 않을때

```md
**중요 내용 추출:**

1. **발신자:** 김철수 (chulsoo.kim@bikecorporation.me)
2. **수신자:** 이은채 (eunchae@teddyinternational.me)
3. **제목:** "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
4. **요청 사항:**
   - ZENESIS 모델의 상세한 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함)
5. **미팅 제안:**
   - 날짜: 다음 주 화요일 (1월 15일)
   - 시간: 오전 10시
   - 장소: 귀사 사무실

6. **발신자 정보:**
   - 김철수, 상무이사, 바이크코퍼레이션
```

<br>

- JSON 형식의 구조화된 답변

```md
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 신규 자전거 'ZENESIS' 모델에 대한 브로슈어 요청과 기술 사양, 배터리 성능, 디자인 정보 요청. 또한, 협력 논의를 위해 1월 15일 오전 10시에 미팅 제안.",
  "date": "1월 15일 오전 10시"
}

```


<br>

## 03-01. Pydantic 출력 파서(`PydanticOutputParser`)

<br>

### `PydanticOuputParser`
- `PydanticOutputParser` 는 언어 모델의 출력을 더 구조화된 정보로 변환 하는 데 도움이 되는 클래스
- 단순 텍스트 형태의 응답 대신, 사용자가 필요로 하는 정보를 명확하고 체계적인 형태로 제공 

- **`PydanticOutputParser` (이는 대부분의 `OutputParser` 에 해당) 에는 주로 두 가지 핵심 메서드가 구현 되어야 함**
    - `get_format_instructions()` : 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침(instruction) 을 제공
      - 예) 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환. 이때 설정하는 지침(`instruction`) 의 역할이 매우 중요
      - 이 지침에 따라 언어 모델은 출력을 구조화하고, 이를 특정 데이터 모델에 맞게 변환
    - `parse()` : 언어 모델의 출력(문자열로 가정)을 받아들여 이를 특정 구조로 분석하고 변환
      - `Pydantic`와 같은 도구를 사용하여, 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환


In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [4]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

<br>

- 이메일 본문 예시

In [5]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

- 출력 파서를 사용하지 않는 경우 예시

In [6]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

In [7]:
prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

for chunk in answer:
    print(chunk.content, flush=True, end='')

이메일의 중요한 내용은 다음과 같습니다:

1. 김철수 상무가 바이크코퍼레이션을 대표하여 이은채 대리에게 연락.
2. "ZENESIS" 자전거에 대한 관심 표명.
3. ZENESIS 모델의 상세한 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함).
4. 유통 전략 및 마케팅 계획 구체화를 위한 정보 필요.
5. 협력 가능성 논의를 위한 미팅 제안: 다음 주 화요일(1월 15일) 오전 10시, 귀사 사무실에서.

<br>

#### `Pydantic` 스타일로 정의된 클래스를 사용하여 이메일의 정보를 파싱

In [8]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [ ]:
parser = PydanticOutputParser(pydantic_object=EmailSummary)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


<br>

- 프롬프트 정의
1. `question` : 유저의 질문
2. `email_conversation` : 이메일 본문의 내용
3. `format` : 형식을 지정

In [13]:
prompt = PromptTemplate.from_template(
    """
    You are a helpful assistant. Please answer the following questions in KOREAN.

    QUESTION:
    {question}

    EMAIL CONVERSATION:
    {email_conversation}

    FORMAT:
    {format}
"""
)

- `PydanticOutputParser의` 부분 포맷팅(partial) 추가

In [14]:
prompt = prompt.partial(format=parser.get_format_instructions())

- chain 생성

In [15]:
chain = prompt | llm

In [31]:
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

In [36]:
output = response.content
print(output)

```json
{
    "person": "김철수",
    "email": "chulsoo.kim@bikecorporation.me",
    "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
    "summary": "김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 제안하며, ZENESIS 모델의 상세한 브로슈어를 요청하고, 협력 가능성을 논의하기 위해 1월 15일 오전 10시에 미팅을 제안합니다.",
    "date": "1월 15일 오전 10시"
}
```


<br>

- `parser를` 사용하여 결과를 파싱하고 `EmailSummary` 객체로 변환

In [38]:
structured_output = parser.parse(output)
print(structured_output)

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary='김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 제안하며, ZENESIS 모델의 상세한 브로슈어를 요청하고, 협력 가능성을 논의하기 위해 1월 15일 오전 10시에 미팅을 제안합니다.' date='1월 15일 오전 10시'


<br>

### `parser` 가 추가된 체인
- 출력 결과를 정의한 Pydantic 객체로 생성


In [39]:
chain = prompt | llm | parser

In [40]:
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무는 바이크코퍼레이션의 자전거 유통 협력을 위해 ZENESIS 모델의 상세한 브로슈어를 요청하고, 협력 가능성을 논의하기 위해 1월 15일 오전 10시에 미팅을 제안합니다.', date='1월 15일 오전 10시')

<br>

### `with_structured_output()`
- `.with_structured_output(Pydantic)`을 사용하여 출력 파서를 추가하면, 출력을 `Pydantic` 객체로 변환 가능
- `.with_structured_output()` 함수는 `stream()` 기능을 지원 X

In [50]:
llm_with_structered = ChatOpenAI(
    temperature=0, model_name="gpt-4o-mini"
).with_structured_output(EmailSummary)

In [51]:
answer = llm_with_structered.invoke(email_conversation)
answer

EmailSummary(person='이은채', email='eunchae@teddyinternational.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary="김철수 상무가 이은채 대리에게 바이크코퍼레이션과 'ZENESIS' 자전거의 유통 협력에 대해 논의하고자 미팅을 제안하며, 제품에 대한 상세한 브로슈어 요청.", date='2024-01-08')

<br>

<hr>

<br>

## 03-02. 콤마 구분자 출력 파서(CommaSeparatedListOutputParser)

<br>

### `CommaSeparatedListOutputParser`
- `CommaSeparatedListOutputParser` 는 쉼표로 구분된 항목 목록을 반환할 필요가 있을 때 유용

- 사용자가 입력한 데이터나 요청한 정보를 쉼표로 구분하여 명확하고 간결한 목록 형태로 제공받을 수 있음
  - 예) 여러 개의 데이터 포인트, 이름, 항목 또는 다른 종류의 값들을 나열할 때 이를 통해 효과적으로 정보를 정리하고 사용자에게 전달
- 이 방법은 정보를 구조화하고, 가독성을 높이며, 특히 데이터를 다루거나 **리스트 형태의 결과를 요구하는 경우**에 매우 유용

In [52]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

- 콤마로 구분된 리스트 출력 파서

In [53]:
output_parser = CommaSeparatedListOutputParser()

- 출력 형식 지침

In [54]:
format_instructions = output_parser.get_format_instructions()

- 주제에 대한 다섯 가지를 나열하라는 템플릿
  - 입력 변수로 `subject` 사용
  - 부분 변수로 형식 지침 사용

In [55]:
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],  
    partial_variables={"format_instructions": format_instructions},
)


In [56]:
model = ChatOpenAI(temperature=0, model='gpt-4o-mini')
chain = prompt | model | output_parser

In [57]:
chain.invoke({"subject": "대한민국 관광명소"})

['경복궁', '남산서울타워', '제주도', '부산 해운대', '경주 불국사']

<br>

- 스트림 출력

In [59]:
for s in chain.stream({"subject": "대한민국 관광명소"}):
    print(s)

['경복궁']
['남산서울타워']
['제주도']
['부산 해운대']
['경주 불국사']


<br>

<hr>

<br>

## 03-03. 구조화된 출력 파서(StructuredOuputParser)
- `StructuredOutputParser`는 LLM에 대한 답변을 `dict` 형식으로 구성하고 **key/value 쌍으로 갖는 여러 필드를 반환하고자 할 때 사용**
    
    **Pydantic/JSON 파서가 더 강력하지만, 이는 덜 강력한 모델(예를 들어 로컬모델과 같은 인텔리전스가 GPT, Claude 모델보다 인텔리전스가 낮은(parameter 수가 낮은) 모델) 에 유용**

    - 로컬 모델은 `Pydantic` 파서가 동작하지 않는 경우가 많으므로, 대안으로 `StructuredOutputParser` 를 사용할 수 있음





In [60]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

- `ResponseSchema` 클래스를 사용하여 사용자의 질문에 대한 답변과 사용된 소스(웹사이트)에 대한 설명을 포함하는 응답 스키마를 정의
- `StructuredOutputParser`를 `response_schemas`를 사용하여 초기화하여, 정의된 응답 스키마에 따라 출력을 구조화

<br>

- 사용자의 질문에 대한 답변

In [61]:
response_schemas = [
    ResponseSchema(
        name="answer", description="사용자의 질문에 대한 답변"
    ),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]

- 응답 스키마를 기반으로 한 구조화된 출력 파서 

In [62]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

- 출력 형식 지시사항을 파싱

In [64]:
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

In [65]:
model = ChatOpenAI(temperature=0, model='gpt-4o-mini')
chain = prompt | model | output_parser

In [66]:
chain.invoke({"question": "대한민국의 수도는 어디인가요?"})

{'answer': '대한민국의 수도는 서울입니다.',
 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8'}

<br>

- 스트리밍

In [67]:
for s in chain.stream({"question": "세종대왕의 업적은 무엇인가요?"}):
    print(s)

{'answer': '세종대왕은 한글을 창제하여 한국어의 표기 체계를 혁신하였고, 과학, 농업, 음악 등 다양한 분야에서 많은 업적을 남겼습니다. 또한, 백성을 위한 정책을 펼쳐 사회의 발전에 기여하였습니다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95'}


<br>

<hr>

<br>

## 03-04. JSON 출력 파서(JsonOutputParser)

<br>

### JsonOutputParser
- 이 출력 파서는 사용자가 원하는 JSON 스키마를 지정할 수 있게 해주며, 그 스키마에 맞게 LLM에서 데이터를 조회하여 결과를 도출
  - LLM이 데이터를 정확하고 효율적으로 처리하여 원하는 형태의 JSON을 생성하기 위해서는, 모델의 용량이 충분해야 함


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [69]:
model = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

- 출력 구조 정의

In [70]:
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [71]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입
parser = JsonOutputParser(pydantic_object=Topic)

In [72]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

In [73]:
prompt = prompt.partial(format_instructions=parser.get_format_instructions())
chain = prompt | model | parser 
chain.invoke({"question": question}) 

{'description': '지구 온난화는 기후 변화의 주요 원인으로, 생태계와 인류에 심각한 영향을 미치고 있습니다.',
 'hashtags': '#지구온난화 #기후변화'}

<br>

### `Pydantic` 없이 사용

In [74]:
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

In [75]:
prompt = prompt.partial(format_instructions=parser.get_format_instructions())
chain = prompt | model | parser
response = chain.invoke({"question": question})

print(response)

{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 온실가스의 증가로 인해 발생합니다. 이로 인해 기후 변화, 해수면 상승, 생태계 변화 등의 문제가 발생할 수 있습니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#지구환경']}


<br>

## 03-05. 데이터프레임 출력 파서(PandasDataFrameOutputParser)

<br>

### `PandasDataFrameOutputParser`
- `Pandas DataFrame`은 Python 프로그래밍 언어에서 널리 사용되는 데이터 구조로, 데이터 조작 및 분석을 위해 흔히 사용
- 구조화된 데이터를 다루기 위한 포괄적인 도구 세트를 제공하여, 데이터 정제, 변환 및 분석과 같은 작업에 다양하게 활용


In [124]:
import pprint
from typing import Any, Dict
import difflib

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [149]:
model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

- 파서 출력을 dict 형식으로 변환하고 출력 형식을 지정하는 함수

In [150]:
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key in parser_output.keys():
        parser_output[key] = parser_output[key].to_dict()

    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

<br>

- `PandasDataFrameOutputParser`를 사용하여 DataFrame을 파싱

In [151]:
df = pd.read_csv("./data/titanic.csv")
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [152]:
parser = PandasDataFrameOutputParser(dataframe=df)
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

- 컬럼에 대한 값을 조회

In [153]:
df_query = "Age column 을 조회해 주세요."

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"], 
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },
)

In [154]:
llm_chain = prompt | model
raw_output = llm_chain.invoke({"query": df_query})

In [156]:
chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})

format_parser_output(parser_output)

{'Age': {0: 22.0,
         1: 38.0,
         2: 26.0,
         3: 35.0,
         4: 35.0,
         5: 27.0,
         6: 54.0,
         7: 2.0,
         8: 27.0,
         9: 14.0,
         10: 4.0,
         11: 58.0,
         12: 20.0,
         13: 39.0,
         14: 14.0,
         15: 55.0,
         16: 2.0,
         17: 23.0,
         18: 31.0,
         19: 22.0,
         20: 35.0,
         21: 34.0,
         22: 15.0,
         23: 28.0,
         24: 8.0,
         25: 38.0,
         26: 26.0,
         27: 19.0,
         28: 24.0,
         29: 23.0,
         30: 40.0,
         31: 48.0,
         32: 18.0,
         33: 66.0,
         34: 28.0,
         35: 42.0,
         36: 18.0,
         37: 21.0,
         38: 18.0,
         39: 14.0,
         40: 40.0,
         41: 27.0,
         42: 3.0,
         43: 19.0,
         44: 30.0,
         45: 20.0,
         46: 27.0,
         47: 16.0,
         48: 18.0,
         49: 7.0,
         50: 21.0,
         51: 49.0,
         52: 29.0,
         

<br>

- 특정 열에서 일부 행의 평균을 검색

In [157]:
df_query = "Retrieve the average of the Ages from row 0 to 4."

parser_output = chain.invoke({"query": df_query})

print(parser_output)

{'mean': 31.2}


<br>

<hr>

<br>

## 03-06. 날짜 형식 출력 파서(DatetimeOutputParser)

<br>

### `DatetimeOutputParser`

<table>
<thead>
<tr>
<th>형식 코드</th>
<th>설명</th>
<th>예시</th>
</tr>
</thead>
<tbody>
<tr>
<td>%Y</td>
<td>4자리 연도</td>
<td>2024</td>
</tr>
<tr>
<td>%y</td>
<td>2자리 연도</td>
<td>24</td>
</tr>
<tr>
<td>%m</td>
<td>2자리 월</td>
<td>07</td>
</tr>
<tr>
<td>%d</td>
<td>2자리 일</td>
<td>04</td>
</tr>
<tr>
<td>%H</td>
<td>24시간제 시간</td>
<td>14</td>
</tr>
<tr>
<td>%I</td>
<td>12시간제 시간</td>
<td>02</td>
</tr>
<tr>
<td>%p</td>
<td>AM 또는 PM</td>
<td>PM</td>
</tr>
<tr>
<td>%M</td>
<td>2자리 분</td>
<td>45</td>
</tr>
<tr>
<td>%S</td>
<td>2자리 초</td>
<td>08</td>
</tr>
<tr>
<td>%f</td>
<td>마이크로초 (6자리)</td>
<td>000123</td>
</tr>
<tr>
<td>%z</td>
<td>UTC 오프셋</td>
<td>+0900</td>
</tr>
<tr>
<td>%Z</td>
<td>시간대 이름</td>
<td>KST</td>
</tr>
<tr>
<td>%a</td>
<td>요일 약어</td>
<td>Thu</td>
</tr>
<tr>
<td>%A</td>
<td>요일 전체</td>
<td>Thursday</td>
</tr>
<tr>
<td>%b</td>
<td>월 약어</td>
<td>Jul</td>
</tr>
<tr>
<td>%B</td>
<td>월 전체</td>
<td>July</td>
</tr>
<tr>
<td>%c</td>
<td>전체 날짜와 시간</td>
<td>Thu Jul  4 14:45:08 2024</td>
</tr>
<tr>
<td>%x</td>
<td>전체 날짜</td>
<td>07/04/24</td>
</tr>
<tr>
<td>%X</td>
<td>전체 시간</td>
<td>14:45:08</td>
</tr>
</tbody>
</table>

In [ ]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [159]:
output_parser = DatetimeOutputParser()
output_parser.format = "%Y-%m-%d"

- 답변 템플릿

In [160]:
template = """Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:"""

In [162]:
prompt = PromptTemplate.from_template(
    template,
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    }, 
)

prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%d'.\n\nExamples: 2025-10-02, 2024-10-02, 2025-10-01\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:')

In [163]:
chain = prompt | ChatOpenAI() | output_parser

output = chain.invoke({"question": "Google 이 창업한 연도는?"})

In [164]:
output.strftime("%Y-%m-%d")

'1998-09-04'

<br>

<hr>

<br>

## 03-07. 열거형 출력 파서(EnumOutputParser)

<br>

### `EnumOutputParser`


In [166]:
from langchain.output_parsers.enum import EnumOutputParser
from enum import Enum

<br>

#### 사용 예시

- `enum` 모듈을 사용하여 `Colors` 클래스를 정의
    - `Colors` 클래스는 `Enum`을 상속받으며, RED, GREEN, BLUE 세 가지 색상 값


In [167]:
class Colors(Enum):
    RED = "빨간색"
    GREEN = "초록색"
    BLUE = "파란색"

- `EnumOutputParser` 인스턴스 

In [168]:
parser = EnumOutputParser(enum=Colors)

- 프롬프트에 사람의 정보(`{person}`)와 파싱 지침(`{instructions}`)을 포함
- `parser.get_format_instructions()` 함수를 호출하여 파싱 지침을 가져옴
- 프롬프트, ChatOpenAI 모델, 파서를 연결하여 처리 체인을 구성

In [169]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [170]:
prompt = PromptTemplate.from_template(
    """다음의 물체는 어떤 색깔인가요?
Object: {object}
Instructions: {instructions}"""
).partial(instructions=parser.get_format_instructions())

chain = prompt | ChatOpenAI() | parser

In [171]:
response = chain.invoke({"object": "하늘"})
print(response)

Colors.BLUE
